Selenium을 이용하여 일간 코로나 현황을 크롤링한 후 csv로 저장하려고 한다.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [2]:
url = 'https://sgis.kostat.go.kr/view/thematicMap/thematicMapMain?stat_thema_map_id=sAXkcVzk5V202007141335257355ued9032uw&theme=CTGR_005&mapType=05'

In [10]:
date_2020 = ['20200923','20200924','20200925','20200926','20200927','20200928','20200929','20200930', '20201001', '20201002', '20201003', '20201004']
Lunar_2021 = ['20210204','20210205','20210206','20210207','20210208','20210209','20210210','20210211', '20210212', '20210213', '20210214']
date_2021 = ['20210911','20210912','20210913','20210914','20210915','20210916','20210917','20210918', '20210919', '20210920', '20210921', '20210922']
Lunar_2022 = ['20220122','20220123','20220124','20220125','20220126','20220127','20220128','20220129','20220130','20220131','20220201','20220202']
date_2022 = ['20220902','20220903','20220904','20220905','20220906','20220907']

In [118]:
empty_data = pd.DataFrame({'date' : [], 'seoul_new_patient' : [], 'busan_new_patient' : [], 'seoul_total_patient' : [], 'busan_total_patient': []})

In [99]:
driver = webdriver.Chrome()
driver.get(url)

In [100]:
iframe = driver.find_element(By.CSS_SELECTOR, "#themeticFrame")
driver.switch_to.frame(iframe)
driver.find_element(By.CSS_SELECTOR, '#corona_board > button').click() # 팝업창 닫기
#driver.find_element(By.CSS_SELECTOR, '#dataBoard > div > div.bar > a').click() # 화면 오른쪽 데이터보드 닫기
#driver.switch_to.default_content()

In [115]:
def SelectDate(date, Time=2):
    year_dict = {2020 : '#select_base_year > option:nth-child(1)',
                2021 : '#select_base_year > option:nth-child(2)',
                2022 : '#select_base_year > option:nth-child(3)'}
    year = date.year
    month = date.month
    day = date.day
    if year <= 2020 and month < 3:
        print('해당 데이터는 2020년 3월부터 입니다. 처음 데이터의 element를 출력합니다.')
        year = 2020
        month = 3
        day = 1
    driver.find_element(By.CSS_SELECTOR, year_dict[year]).click()
    time.sleep(Time)
    if date.year < 2021:
        driver.find_element(By.CSS_SELECTOR, f'#covid_month > option:nth-child({month-2})').click()
    else:
        driver.find_element(By.XPATH, f'//*[@id="covid_month"]/option[{month}]').click()
    time.sleep(Time)
    driver.find_element(By.XPATH, f'//*[@id="covid_day"]/option[{day}]').click()

In [127]:
def makeDataFrame(date_list):
    data = empty_data.copy()
    for str_date in date_list:
        key = []
        value = []
        date = datetime.strptime(str_date, '%Y%m%d').date()
        key.append('date'); value.append(date);
        SelectDate(date)
        driver.find_element(By.CSS_SELECTOR, '#leftValue').click() # 신규(명) 으로 전환.
        time.sleep(3)
        # 서울
        s_new_patient = driver.find_element(By.CSS_SELECTOR, \
                                                '#mapRgn_1 > div > div.sop-pane.sop-overlay-pane > div:nth-child(3) > span').text
        # 부산
        b_new_patient = driver.find_element(By.CSS_SELECTOR, \
                                                '#mapRgn_1 > div > div.sop-pane.sop-overlay-pane > div:nth-child(4) > span').text
        key.append('seoul_new_patient'); value.append(s_new_patient)
        key.append('busan_new_patient'); value.append(b_new_patient)
        driver.find_element(By.CSS_SELECTOR, '#rightValue').click() # 누적(명) 으로 전환.
        time.sleep(3)
        s_total_patient = driver.find_element(By.CSS_SELECTOR, \
                                                '#mapRgn_1 > div > div.sop-pane.sop-overlay-pane > div:nth-child(3) > span').text
        b_total_patient = driver.find_element(By.CSS_SELECTOR, \
                                                '#mapRgn_1 > div > div.sop-pane.sop-overlay-pane > div:nth-child(4) > span').text
        key.append('seoul_total_patient'); value.append(s_total_patient)
        key.append('busan_total_patient'); value.append(b_total_patient)
        
        my_dict = dict(zip(key, value))
        df = pd.DataFrame(my_dict, index = [0])
        data = pd.concat([data, df], axis=0)
        data = data.astype({'seoul_new_patient' : 'int',
                      'busan_new_patient' : 'int',
                     'seoul_total_patient' : 'int',
                     'busan_total_patient' : 'int'})
    return data

In [132]:
data20 = makeDataFrame(date_2020)
Lunar21 = makeDataFrame(Lunar_2021)
data21 = makeDataFrame(date_2021)
Lunar22 = makeDataFrame(Lunar_2022)
data22 = makeDataFrame(date_2022)

In [133]:
data = pd.concat([data20, Lunar21, data21, Lunar22, data22], axis=0)

In [134]:
data.shape

(53, 5)

In [135]:
data

,date,seoul_new_patient,busan_new_patient,seoul_total_patient,busan_total_patient
0,2020-09-23,40,7,5056,397
0,2020-09-24,39,7,5095,404
0,2020-09-25,57,3,5152,407
0,2020-09-26,26,2,5178,409
0,2020-09-27,34,3,5212,412
0,2020-09-28,19,1,5231,413
0,2020-09-29,11,3,5242,416
0,2020-09-30,51,5,5293,421
0,2020-10-01,30,6,5323,427
0,2020-10-02,11,18,5334,445


In [136]:
data.to_csv('corona_19.csv', index=False)

In [137]:
driver.quit()